# Calculate utilities of two job decisions.


In [1]:
import numpyro
import numpyro.distributions as dist
from jax import random
import plotly.express as px

/home/justin/bin/mambaforge/envs/job_decision/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rng_key = random.PRNGKey(0)

In [3]:
# https://realpython.com/python-super/#an-overview-of-pythons-super-function
class Pert(dist.Beta):
    def __init__(self, a, b, c, gamma):
        # https://pubsonline.informs.org/doi/epdf/10.1287/ited.1080.0013
        # Davis 2008 formula for conversion
        # between PERT and beta distributions
        # https://en.wikipedia.org/wiki/PERT_distribution#The_modified-PERT_distribution
        mu = (a + gamma * b + c)/(gamma + 2)
        # https://reference.wolfram.com/language/ref/PERTDistribution.html
        simga_squared = (c - a - b*gamma + c*gamma)*(c + b*gamma - a * (1 + gamma))/((2 + gamma)**2 * (3 + gamma))
        part_0 = 3*(c - a)
        part_1 = 1 + 4*(b - a)*(c - b)/(c - a)**2
        part_2 = part_1 / part_0
        concentration1 = 2*(c + 4*b - 5*a)*part_2
        concentration0 = 2*(5*c - 4*b - a)*part_2
        super().__init__(concentration1, concentration0)

In [26]:
a = 0
b = 10
c = 66
pert = Pert(a, b, c)

In [27]:
# https://num.pyro.ai/en/0.7.1/getting_started.html
y = a + pert.sample(key=rng_key, sample_shape=(100_000,))*(c - a)

In [28]:
fig = px.histogram(y)
fig.show()